In [1]:
import tensorflow.keras
import numpy as np
import os
import json
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

os.chdir("/workspace/telemed5000/ecg/ecg")
import network
import load
from MITDBDataProvider import *
os.chdir('../')

# The cool stuff

In [2]:
def average_weights(models):
    weights = [model.get_weights() for model in models]
    new_weights = list()

    for weights_list_tuple in zip(*weights):
        new_weights.append(
            np.array([np.array(weights_).mean(axis=0)\
                for weights_ in zip(*weights_list_tuple)]))
    return new_weights

In [3]:
params = json.load(open("examples/cinc17/config.json", 'r'))
params.update({
    "input_shape": [None, 1],
    "num_categories": 4
})
def create_model():
    with tf.device('/gpu:2'):
        model = network.build_network(**params)
        optimizer = Adam(
            lr=params["learning_rate"],
            clipnorm=params.get("clipnorm", 1))

        model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
        return model

def fit_model(model, index, dataLoader):
    with tf.device('/gpu:2'):
        train_x, train_y = dataLoader.clientData(index)
        model.fit(train_x, train_y)
        return model

In [4]:
def fit_federated(clients_count, federated_epochs, loader):
    models = list()
    print("Starting to create models...")
    for i in range(clients_count):
        model = create_model()
        models.append(model)
        
    print("Starting the training...")

    for epoch in range(federated_epochs):
        for index, model in enumerate(models):      
            fit_model(model, index, loader)
        new_weights = average_weights(models)
        for model in models:
            model.set_weights(new_weights)
        # score = model.evaluate(loader.validationData[0], loader.validationData[1], verbose=0)
        # print('Test loss:', score[0])
        # print('Test accuracy:', score[1])
        return model

# Data Loader

In [5]:
class CINCFederatedDataLoader:
    
    def __init__(self, train_path, validation_path, clients):
        plain_data = load.load_dataset(train_path)
        self.allData = load.Preproc(*plain_data).process(*plain_data)
        self.clients = clients
        self.groupClientData()
        plain_validation_data = load.load_dataset(validation_path)
        preproc = load.Preproc(*plain_validation_data)
        self.validationData = preproc.process(*plain_validation_data)
    
    def clientData(self, index):
        return self.clientsData[index]
    
    def groupClientData(self):
        self.clientsData = []
        xSplit = self.splitArray(self.allData[0])
        ySplit = self.splitArray(self.allData[1])
        for index in range(self.clients):
            self.clientsData.append([xSplit[index], ySplit[index]])

    def splitArray(self, array):
        return np.array_split(array, self.clients)

In [6]:
class MITFederatedDataLoader:
    
    def __init__(self, file_path):
        # x train x test y train y test
        rawData = get_data_4_fed(file_path)
        self.x_train = self.prepare_x_train(rawData[0])
        self.x_test = self.prepare_x(rawData[1])
        self.y_train = self.prepare_y_train(rawData[2])
        self.y_test = self.prepare_y(rawData[3])
        #self.validationData = [np.expand_dims(np.array(self.x_test), axis=2), np.array(self.y_test)]
        self.validationData = [np.array(self.x_test), np.array(self.y_test)]
    def prepare_y(self, data_chunk):
        return [np.tile(y_1, (168, 1)) for y_1 in data_chunk]

    def prepare_y_train(self, data):
        return [self.prepare_y(y_1) for y_1 in data]
    
    def prepare_x(self, data_chunk):
        return [x_1[:len(x_1)-192] for x_1 in data_chunk]
    
    def prepare_x_train(self, data):
        return [self.prepare_x(x_1) for x_1 in data]
        
    def clientData(self, index):
        return [np.expand_dims(np.array(self.x_train[index]),axis=2), np.array(self.y_train[index])]

# Fit cinc17

In [37]:
data_json = "examples/cinc17/train.json"
cincLoader = CINCFederatedDataLoader(data_json, params['dev'], 5)
#cincModel = fit_federated(5, 1, cincLoader)
cincLoader.clientData(1)[0].shape

100%|██████████| 852/852 [00:00<00:00, 4335.45it/s]


(1535, 18176, 1)

# Fit MITDB

In [7]:
file_path = "/workspace/telemed5000/code/data/"
loader = MITFederatedDataLoader(file_path)

# mITModel = fit_federated(47, 200, loader)
mITModel = fit_federated(2, 1, loader)

100%|██████████| 48/48 [00:07<00:00,  6.78it/s]


Starting to create models...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Starting the training...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
9/9 [==============================] - 14s 2s/sample - loss: 1.9409 - acc: 0.0813


In [98]:
# print(cincLoader.clientData(1)[1][0]
# y_1 = loader.y_train

In [ ]:
loader = MITFederatedDataLoader(file_path)

score = mITModel.evaluate(loader.validationData[0], loader.validationData[1], verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [81]:
loader.clientData(5)

[array([[[-0.26 ],
         [-0.295],
         [-0.325],
         ...,
         [-0.375],
         [-0.36 ],
         [-0.34 ]],
 
        [[-0.45 ],
         [-0.445],
         [-0.43 ],
         ...,
         [-0.19 ],
         [-0.2  ],
         [-0.23 ]],
 
        [[-0.225],
         [-0.22 ],
         [-0.22 ],
         ...,
         [-0.34 ],
         [-0.325],
         [-0.325]],
 
        ...,
 
        [[-0.225],
         [-0.23 ],
         [-0.245],
         ...,
         [-0.12 ],
         [-0.1  ],
         [-0.1  ]],
 
        [[-0.335],
         [-0.345],
         [-0.34 ],
         ...,
         [-0.265],
         [-0.275],
         [-0.275]],
 
        [[-0.445],
         [-0.445],
         [-0.445],
         ...,
         [-0.25 ],
         [-0.255],
         [-0.24 ]]]), array([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0

In [197]:
loader.prepare_y(loader.y)

TypeError: 'NoneType' object is not iterable

In [200]:
y2 = np.load('ecg/MIT/' + 'Y_MIT.npy')

In [201]:
np.squeeze(y2)

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=int32)

In [210]:
y = np.array()
for _y in y2:
    _y = np.append(_y, [0, 0])
    print(_y)
    y = np.concatenate(y, _y)

TypeError: Required argument 'object' (pos 1) not found

100%|██████████| 48/48 [00:03<00:00, 15.07it/s]


([[array([-0.145, -0.145, -0.145, ..., -0.38 , -0.38 , -0.36 ]),
   array([-0.345, -0.35 , -0.34 , ..., -0.36 , -0.35 , -0.355]),
   array([-0.29 , -0.285, -0.265, ..., -0.365, -0.38 , -0.375]),
   array([-0.31 , -0.325, -0.33 , ..., -0.2  , -0.225, -0.235]),
   array([-0.38, -0.39, -0.4 , ..., -0.43, -0.44, -0.43]),
   array([-0.365, -0.37 , -0.385, ..., -0.3  , -0.31 , -0.3  ]),
   array([-0.365, -0.375, -0.385, ..., -0.42 , -0.42 , -0.425]),
   array([-0.43 , -0.425, -0.42 , ..., -0.36 , -0.36 , -0.345]),
   array([-0.27 , -0.275, -0.28 , ..., -0.305, -0.3  , -0.3  ])],
  [array([-0.32 , -0.335, -0.33 , ..., -0.345, -0.345, -0.34 ]),
   array([-0.215, -0.215, -0.21 , ..., -0.33 , -0.315, -0.31 ]),
   array([ 0.205,  0.24 ,  0.26 , ..., -0.63 , -0.635, -0.62 ]),
   array([-0.605, -0.6  , -0.62 , ..., -0.37 , -0.365, -0.37 ]),
   array([-0.32 , -0.325, -0.325, ..., -0.46 , -0.445, -0.445]),
   array([-0.345, -0.345, -0.345, ...,  0.22 ,  0.205,  0.2  ]),
   array([-0.14 , -0.14 , -0.1